In [1]:
import json
import math
import os
import random
import re
import time
from datetime import datetime
from string import punctuation, whitespace

import emoji
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
from IPython.display import clear_output, display
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.common.exceptions import WebDriverException
from selenium.webdriver.common.by import By
from webdriver_manager.chrome import ChromeDriverManager
# from seleniumwire import webdriver
import pickle
import time
from ast import literal_eval
from selenium_stealth import stealth
import undetected_chromedriver as uc
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import random

In [2]:
approximate_reviews_count = 150 
product_counts = 15
sleep_time = 1
reviews_per_load = 30
waiting_time_product = 1
driver_wait_time = 3

scrolls_count = approximate_reviews_count / 1.2

In [3]:
def get_category_links(driver, group_url):
    driver.get(group_url)
    # Раскрыть менюшку категорий
    dropdown = driver.find_elements(
        By.CSS_SELECTOR, ".dropdown-filter__btn.dropdown-filter__btn--burger"
    )
    dropdown[-1].click()
    # Спарсить ссылки на конечные категории (спарсить листья дерева)
    elements_categories = driver.find_elements(
        By.CSS_SELECTOR, ".filter-category__link.j-category-forward.j-category"
    )
    links = [elem.get_attribute("href") for elem in elements_categories]
    return links

In [4]:
def get_product_links_category_name(
    driver, category_link, waiting_time_product, product_counts
):
    # driver = webdriver.Chrome(options=chrome_options)
    driver.get(category_link)

    category_name_element = driver.find_elements(By.CSS_SELECTOR, ".breadcrumbs__item")
    category_name = " / ".join([cn.text for cn in category_name_element[1:]])
    category_products = driver.find_elements(
        By.CSS_SELECTOR, ".product-card__link.j-card-link.j-open-full-product-card"
    )
    products_urls = [elem.get_attribute("href") for elem in category_products]
    time.sleep(waiting_time_product)
    # print(products_urls)
    product_ids = [re.findall(r"(\bcatalog\b)\/(\d+)", u)[0][1] for u in products_urls][
        :product_counts
    ]
    # print(product_ids)
    return (
        [f"https://www.wildberries.ru/catalog/{id}/feedbacks" for id in product_ids],
        category_name,
    )

In [5]:
def scroll_parse_reviews(
    driver,
    url,
    scrolls_count = 100,
    approximate_reviews_count=100,
    sleep_time=0.5,
    waiting_time_product=10,
    reviews_per_load=10,
    category="unknown",
):
    res = []
    refresh_times = math.ceil(approximate_reviews_count / reviews_per_load)

    driver.get(url)
    # Сортируем отзыввы по полезности
    
    try:
        WebDriverWait(driver, 60).until(
            EC.presence_of_element_located((By.CSS_SELECTOR, '.product-line__name.hide-mobile'))
        )
    except:
        pass
    elements_categories = driver.find_elements(By.CLASS_NAME, "sorting__item")
    driver.implicitly_wait(1)
    if elements_categories:
        elements_categories[-1].click()
    try:
        driver.find_elements(By.CLASS_NAME, "product-feedbacks__title")[-1].click()
    except:
        pass
    # Подгружаем отзывы
    for i in range(int(scrolls_count)):
        driver.execute_script(f"window.scrollBy(0, {random.randrange(200,400)});")
        time.sleep(random.randrange(1,4)/100)
    # for i in range(refresh_times):
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    #     time.sleep(sleep_time)
    driver.implicitly_wait(driver_wait_time)
    # Парсим подгруженную страницу отзывов
    html = driver.page_source
    soup = BeautifulSoup(html)
    reviews = soup.findAll("li", class_="comments__item feedback j-feedback-slide")
    # product_name = soup.find("a", class_="product-line__name hide-mobile")
    
    # reviews = [item.text for item in driver.find_elements(By.CSS_SELECTOR, '.comments__item.feedback.j-feedback-slide')]
    product_name = driver.find_element(By.CSS_SELECTOR, '.product-line__name.hide-mobile').text
    
    
    for review in reviews:
        # 
        # 
        # Всё парсит, но вставляет скрытый спан с текстом "ещё" в конце
        # 
        # 
        text = review.find("p", class_="feedback__text")
        if text:
            text = ''.join(text.find_all(text=True, recursive=False))
        title_element = review.find(
            "span",
            class_=lambda value: value
            and value.startswith("feedback__rating stars-line"),
        )
        if text:
            res.append(
                {
                    "review_full_text": text,
                    "review_rating": title_element["class"][-1][-1],
                    "product": product_name,
                    "category": category,
                    "url": driver.current_url,
                }
            )
    soup.decompose()
    time.sleep(waiting_time_product)
    return res

In [6]:
from fake_useragent import UserAgent

chrome_options = Options()

# chrome_options.add_argument("user-agent=Chrome/91.0.4472.124")
chrome_options.add_argument("--disable-blink-features")
chrome_options.add_argument("--disable-blink-features=AutomationControlled")
chrome_options.add_argument("--disable-extensions")
chrome_options.add_argument("--no-sandbox")
chrome_options.add_argument("--start-maximized")
chrome_options.add_argument("--disable-dev-shm-usage")
chrome_options.add_argument("--incognito")
chrome_options.add_argument("disable-infobars")
chrome_options.add_experimental_option("excludeSwitches", ["enable-logging"])
chrome_options.add_experimental_option("useAutomationExtension", False)
chrome_options.add_experimental_option("excludeSwitches", ["enable-automation"])
chrome_options.add_experimental_option("detach", True)

chrome_options.add_argument('window-size=1920x1080')
# chrome_options.add_argument("disable-gpu")

chrome_options.add_argument("--disable-popup-blocking")
chrome_options.add_argument("--ignore-certificate-errors")
chrome_options.add_argument("--disable-software-rasterizer")
chrome_options.add_argument("--disable-background-timer-throttling")
chrome_options.add_argument("--disable-backgrounding-occluded-windows")

chrome_options.page_load_strategy = 'eager'


chrome_options.add_argument('headless')

In [7]:
proxy_host = '194.28.192.129'  # Прокси хост
proxy_port = '8000'  # Прокси порт
proxy_user = 'TtQ9VK'  # Прокси логин
proxy_pass = 'uVPw7X'  # Прокси пароль

In [8]:
log_path = "./selenium.log"

In [9]:
# options = {
#    'proxy': {
#         'http': 'socks5://' + proxy_user + ':' + proxy_pass + '@' + proxy_host + ':'+proxy_port,
#         'https': 'socks5://' + proxy_user + ':' + proxy_pass + '@' + proxy_host + ':'+proxy_port,
#         'no_proxy': 'localhost,127.0.0.1'
#     }
# }

In [10]:
driver = webdriver.Chrome(
    options = chrome_options
)
driver.implicitly_wait(driver_wait_time)
stealth(driver,
        languages=["en-US", "en"],
        vendor="Google Inc.",
        platform="Win32",
        webgl_vendor="Intel Inc.",
        renderer="Intel Iris OpenGL Engine",
        fix_hairline=True)
driver.get("https://www.wildberries.ru/catalog/158778298/feedbacks")

In [11]:
# driver.get("https://whatismyipaddress.com")

In [12]:
url = "https://static-basket-01.wbbasket.ru/vol0/data/main-menu-ru-ru-v2.json"

response = requests.get(url)

if response.status_code == 200:
    data = response.json()
else:
    print(f"Ошибка: {response.status_code}")

In [13]:
# Функция для рекурсивного извлечения всех записей
def extract_records(data, parent_id=None, category=""):
    records = []
    for item in data:
        record = {key: value for key, value in item.items() if key != 'childs'}
        record['parent_id'] = parent_id
        record['category'] = "/".join([category, record['name']])
        records.append(record)
        if 'childs' in item:
            try:
                records.extend(extract_records(item['childs'], parent_id=item['id'], category=record['category']))
            except:
                k = 0
    return records

# Извлечение всех записей
all_records = extract_records(data)

In [14]:
df_categories = pd.DataFrame.from_dict(all_records)
df_categories

,id,name,url,parent_id,category,shard,query,isDenyLink,parent,seo
0,1111,ЖАР-СКИДКИ,/promotions/zhar-dni-letnyaya-rasprodazha,NaN,/ЖАР-СКИДКИ,NaN,NaN,NaN,NaN,NaN
1,1234,Сертификаты Wildberries,/gift/certificates,NaN,/Сертификаты Wildberries,NaN,NaN,NaN,NaN,NaN
2,306,Женщинам,/catalog/zhenshchinam,NaN,/Женщинам,blackhole,cat=306,True,NaN,NaN
3,8126,Блузки и рубашки,/catalog/zhenshchinam/odezhda/bluzki-i-rubashki,306.0,/Женщинам/Блузки и рубашки,bl_shirts,cat=8126,NaN,306.0,Женские блузки и рубашки
4,8127,Брюки,/catalog/zhenshchinam/odezhda/bryuki-i-shorty,306.0,/Женщинам/Брюки,pants,cat=8127,NaN,306.0,Женские брюки
...,...,...,...,...,...,...,...,...,...,...
2202,130630,Автобусы,https://vmeste.wildberries.ru/bus?utm_source=w...,61037.0,/Путешествия/Автобусы,NaN,NaN,NaN,61037.0,NaN
2203,130631,Маршруты,https://vmeste.wildberries.ru/ammp?utm_source=...,61037.0,/Путешествия/Маршруты,NaN,NaN,NaN,61037.0,NaN
2204,130851,Отели,https://vmeste.wildberries.ru/hotel?utm_source...,61037.0,/Путешествия/Отели,NaN,NaN,NaN,61037.0,NaN
2205,130628,Регионы,https://vmeste.wildberries.ru/adventure,61037.0,/Путешествия/Регионы,NaN,NaN,NaN,61037.0,NaN


In [15]:
def add_substring_if_missing(s):
    if "http" not in s:
        return "https://www.wildberries.ru" + s
    return s

# Применение функции к столбцу 'text'
df_categories['url'] = df_categories['url'].apply(add_substring_if_missing)
df_categories

,id,name,url,parent_id,category,shard,query,isDenyLink,parent,seo
0,1111,ЖАР-СКИДКИ,https://www.wildberries.ru/promotions/zhar-dni...,NaN,/ЖАР-СКИДКИ,NaN,NaN,NaN,NaN,NaN
1,1234,Сертификаты Wildberries,https://www.wildberries.ru/gift/certificates,NaN,/Сертификаты Wildberries,NaN,NaN,NaN,NaN,NaN
2,306,Женщинам,https://www.wildberries.ru/catalog/zhenshchinam,NaN,/Женщинам,blackhole,cat=306,True,NaN,NaN
3,8126,Блузки и рубашки,https://www.wildberries.ru/catalog/zhenshchina...,306.0,/Женщинам/Блузки и рубашки,bl_shirts,cat=8126,NaN,306.0,Женские блузки и рубашки
4,8127,Брюки,https://www.wildberries.ru/catalog/zhenshchina...,306.0,/Женщинам/Брюки,pants,cat=8127,NaN,306.0,Женские брюки
...,...,...,...,...,...,...,...,...,...,...
2202,130630,Автобусы,https://vmeste.wildberries.ru/bus?utm_source=w...,61037.0,/Путешествия/Автобусы,NaN,NaN,NaN,61037.0,NaN
2203,130631,Маршруты,https://vmeste.wildberries.ru/ammp?utm_source=...,61037.0,/Путешествия/Маршруты,NaN,NaN,NaN,61037.0,NaN
2204,130851,Отели,https://vmeste.wildberries.ru/hotel?utm_source...,61037.0,/Путешествия/Отели,NaN,NaN,NaN,61037.0,NaN
2205,130628,Регионы,https://vmeste.wildberries.ru/adventure,61037.0,/Путешествия/Регионы,NaN,NaN,NaN,61037.0,NaN


In [16]:
import urllib.parse


def encode_forbidden_chars(folder_name):
    forbidden_chars = '<>:"/\\|?*'
    encoded_name = ""
    for char in folder_name:
        if char in forbidden_chars:
            encoded_name += urllib.parse.quote(char)
        else:
            encoded_name += char
    return encoded_name


# for folder_name in ["./wbdirs" + item for item in list(df_categories["category"])][:1]:
#     encoded_folder_name = encode_forbidden_chars(folder_name)
#     os.makedirs(encoded_folder_name, exist_ok=True)

In [17]:
to_wait = (
    ((waiting_time_product + scrolls_count * 0.025 * 1.1))
    * product_counts
    * len(df_categories)
)

for i, df_row in df_categories.iterrows():
    all_reviews = []
    try:
        links, category_name = get_product_links_category_name(
            driver, df_row["url"], waiting_time_product, product_counts
        )
        for url in links:
            reviews = scroll_parse_reviews(
                driver,
                url,
                scrolls_count,
                approximate_reviews_count,
                sleep_time,
                waiting_time_product,
                reviews_per_load,
                category=category_name,
            )
            if reviews:
                all_reviews.extend(reviews)
                clear_output(wait=True)
                display(f"Approximate waiting time: {round(to_wait)}s")
                display(f"Parsed reviews: {len(all_reviews)}")
                display(f"Current category: {df_row['category']}")
                display(f"Categories left: {len(df_categories) - (i + 1)}")
                display(f"Categories parsed: {i + 1}")

        folder_name = "./wbdirs" + df_row["category"]
        encoded_folder_name = encode_forbidden_chars(folder_name)
        os.makedirs(encoded_folder_name, exist_ok=True)
        if all_reviews:
            df = pd.DataFrame.from_dict(all_reviews)
            df["category"] = df_row["category"]
            dt = datetime.utcnow().strftime("%Y-%m-%d-%H-%M-%S")
            file_path = os.path.join(
                os.path.abspath(encoded_folder_name), f"unprepared_{dt}.csv"
            )
            df.to_csv(file_path, encoding="UTF-8", index=False)
        else:
            file_path = os.path.join(os.path.abspath(encoded_folder_name), ".empty")
            with open(file_path, "w") as file:
                file.write("empty")

    except Exception as e:
        print(e)
    all_reviews = []
# 177

'Approximate waiting time: 146903s'

'Parsed reviews: 103'

'Current category: /Детям/Для девочек/Пиджаки и жакеты'

'Categories left: 2030'

'Categories parsed: 177'

Message: no such element: Unable to locate element: {"method":"css selector","selector":".product-line__name.hide-mobile"}
  (Session info: chrome-headless-shell=126.0.6478.127); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF64D4AEEA2+31554]
	(No symbol) [0x00007FF64D427ED9]
	(No symbol) [0x00007FF64D2E872A]
	(No symbol) [0x00007FF64D338434]
	(No symbol) [0x00007FF64D33853C]
	(No symbol) [0x00007FF64D37F6A7]
	(No symbol) [0x00007FF64D35D06F]
	(No symbol) [0x00007FF64D37C977]
	(No symbol) [0x00007FF64D35CDD3]
	(No symbol) [0x00007FF64D32A33B]
	(No symbol) [0x00007FF64D32AED1]
	GetHandleVerifier [0x00007FF64D7B8B1D+3217341]
	GetHandleVerifier [0x00007FF64D805AE3+3532675]
	GetHandleVerifier [0x00007FF64D7FB0E0+3489152]
	GetHandleVerifier [0x00007FF64D55E776+750614]
	(No symbol) [0x00007FF64D43375F]
	(No symbol) [0x00007FF64D42EB14]
	(No symbol) [0x000

In [ ]:
pd.read_csv("./wbdirs/ЖАР-СКИДКИ/unprepared_2024-07-15-10-36-26.csv")

,review_full_text,review_rating,product,category,url
0,"Пришли раньше времени, что меня очень порадова...",5,HUGGIES / Подгузники для новорожденных Elite S...,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/17358082/fe...
1,Заказала для изготовления торта из подгузников...,5,HUGGIES / Подгузники для новорожденных Elite S...,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/17358082/fe...
2,"Купила зелёные памперсы от фирмы памперс,испор...",5,HUGGIES / Подгузники для новорожденных Elite S...,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/17358082/fe...
3,"Спасибо, всё хорошо) дополнительно упакованы в...",5,HUGGIES / Подгузники для новорожденных Elite S...,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/17358082/fe...
4,Придумайте что нибудь с дополнительной упаковк...,5,HUGGIES / Подгузники для новорожденных Elite S...,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/17358082/fe...
...,...,...,...,...,...
2012,"Добрый день, пылесос собирает кошачью шерсть н...",5,Deerma / Пылесос вертикальный компактный для дома,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/68879102/fe...
2013,Один раз использовали в деревне на даче. Все п...,5,Deerma / Пылесос вертикальный компактный для дома,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/68879102/fe...
2014,"Маленький, аккуратненький, достаточно мощный. ...",5,Deerma / Пылесос вертикальный компактный для дома,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/68879102/fe...
2015,"Громкий очень, но терпимо. Когда работает из н...",4,Deerma / Пылесос вертикальный компактный для дома,/ЖАР-СКИДКИ,https://www.wildberries.ru/catalog/68879102/fe...


In [ ]:
# Если надо разделть на разные датафреймы для каждого товара
gb = df.groupby('product')    
temp_gb = [gb.get_group(x) for x in gb.groups]
temp_gb[0]

,review_full_text,review_rating,product,category,url
9513,"Всё отлично. Пришла быстро, три режима, работа...",5,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9514,Всё суперещё,5,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9515,"Не работает автоматическое вкл/выкл, очень жал...",3,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9516,Все работаетещё,5,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9517,"Лампа хорошая есть режим работы, очень удоб...",5,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
...,...,...,...,...,...
9608,"Качество так себе, ещё тысячу стоятещё",1,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9609,Отличная лампа.ещё,5,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9610,Нет адаптера в комплектеещё,4,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
9611,2 лампы а не триещё,2,AMBrand / Фитолампа для растений и рассады све...,"Сад и дача / Горшки, опоры и все для рассады /...",https://www.wildberries.ru/catalog/139425161/f...
